## 数据预处理

In [1]:
import os
import cv2
import h5py
import numpy as np
import pandas as pd
from keras.applications import Xception, xception
from keras.models import Model, Sequential
from keras.layers import Dropout, Dense, Input, Lambda, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
data_path_train = './dataset-mini-30-15/train'
data_path_train_extra = 'dataset/images-Oxford-IIIT2'
data_path_test = './dataset-mini-30-15/test'
image_names_train = os.listdir(data_path_train)
image_names_train_extra = os.listdir(data_path_train_extra)
image_names_test = os.listdir(data_path_test)
input_shape = (224, 224, 3)
labels = []
trains = []
tests = []


# 处理标准的训练数据
for i in tqdm(range(len(image_names_train))):
    image_name = image_names_train[i]
    image_path = os.path.join(data_path_train, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print('Read train image failed:', image_path)
        continue
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    image = image / 255
    trains.append(image[:, :, ::-1])
    # cat: 0, dog: 1
    category = 1 if 'dog' in image_name else 0
    labels.append(category)

    
# 猫的种类
cat_types = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair', 'Egyptian_Mau', 'Maine_Coon', 'Persian',
             'Ragdoll', 'Russian_Blue', 'Siamese', 'Sphynx']
# 处理扩展的训练数据
for i in tqdm(range(len(image_names_train_extra))):
    image_name = image_names_train_extra[i]
    image_path = os.path.join(data_path_train_extra, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print('Read extra train image failed:', image_path)
        continue
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    image = image / 255
    index = len(image_names_train) + i
    trains.append(image[:, :, ::-1])
    
    # 获取动物的种类（dog or cat）
    spt = image_names_train_extra[i].split('_')
    spt.pop()
    tp = '_'.join(spt)
    category = 0 if tp in cat_types else 1
    labels.append(category)
    

# 处理标准的测试数据
for i in tqdm(range(len(image_names_test))):
    image_name = image_names_test[i]
    image_path = os.path.join(data_path_test, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print('Read test image failed:', image_path)
        continue
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    image = image / 255
    tests.append(image[:, :, ::-1])
    
    
trains = np.array(trains)
labels = np.array(labels)
tests = np.array(tests)

print('Training data size: %d' % len(trains))
print('Label size: %d' % len(labels))
print('Testing data size: %d' % len(tests))

100%|██████████| 15/15 [00:00<00:00, 198.65it/s]


Read extra train image failed: dataset/images-Oxford-IIIT2/Egyptian_Mau_167.jpg
Read extra train image failed: dataset/images-Oxford-IIIT2/Abyssinian_100.mat
Read extra train image failed: dataset/images-Oxford-IIIT2/Abyssinian_101.mat
Training data size: 37
Label size: 37
Testing data size: 15


## 构建模型

In [3]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

print('Model ready!')

Model ready!


## 训练

In [4]:
X_train, X_val, y_train, y_val = train_test_split(trains, labels, shuffle=True, test_size=0.2, random_state=2018)

model.fit(x=X_train, y=y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Train on 29 samples, validate on 8 samples
Epoch 1/10
29/29 [==============================] - 39s 1s/step - loss: 0.6935 - acc: 0.4138 - val_loss: 0.6932 - val_acc: 0.3750
Epoch 2/10
29/29 [==============================] - 40s 1s/step - loss: 0.6939 - acc: 0.3448 - val_loss: 0.6933 - val_acc: 0.3750
Epoch 3/10
29/29 [==============================] - 38s 1s/step - loss: 0.6932 - acc: 0.4828 - val_loss: 0.6935 - val_acc: 0.3750
Epoch 4/10
29/29 [==============================] - 68s 2s/step - loss: 0.6937 - acc: 0.4483 - val_loss: 0.6937 - val_acc: 0.3750
Epoch 5/10
29/29 [==============================] - 40s 1s/step - loss: 0.6934 - acc: 0.4483 - val_loss: 0.6939 - val_acc: 0.3750
Epoch 6/10
29/29 [==============================] - 40s 1s/step - loss: 0.6924 - acc: 0.6207 - val_loss: 0.6941 - val_acc: 0.3750
Epoch 7/10
29/29 [==============================] - 43s 1s/step - loss: 0.6935 - acc: 0.4828 - val_loss: 0.6943 - val_acc: 0.3750
Epoch 8/10
29/29 [=============================

## 测试

In [6]:
y_pred = model.predict(tests, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

df = pd.read_csv("sample_submission.csv")

for i in range(len(image_names_test)):
    image_name = image_names_test[i]
    index = int(str.split(image_name, '.')[0]) - 1
    df.iat[index, 1] = y_pred[i]

df.to_csv('predict.csv', index=None)
print('The prediction result has been wrote to predict.csv')

15/15 [==============================] - 6s 372ms/step
The prediction result has been wrote to predict.csv
